In [68]:
import numpy as np
import math 
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
#nltk.download('punkt')   # one time execution
#nltk.download('stopwords')  # one time execution

In [70]:
import grpc
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
import zemberek_grpc.normalization_pb2 as z_normalization
import zemberek_grpc.normalization_pb2_grpc as z_normalization_g
channel = grpc.insecure_channel('localhost:6789')
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)
normalization_stub = z_normalization_g.NormalizationServiceStub(channel)
#java -jar zemberek-full.jar StartGrpcServer --dataRoot .\zemberek_data\

In [71]:
def textRankSimilarities(i,j):
    sum = 0
    for w in i.split():
        if w in j.split():
            sum += 1
    return sum/(math.log(len(i.split()))+math.log(len(j.split())))


In [72]:
def cvtfidfCosineSim(stemmedSen,alg = "cv"):
    if(alg == "cv"):
        CountVec = CountVectorizer(ngram_range=(1,1))
        Count_data = CountVec.fit_transform(stemmedSen)
        cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())  
    else:
        CountVec = TfidfVectorizer()
        Count_data = CountVec.fit_transform(stemmedSen)
        cv_dataframe = pd.DataFrame(Count_data.toarray(), columns=CountVec.get_feature_names())  
    
    sim_mat = np.zeros([len(stemmedSen), len(stemmedSen)])
    
    for i in range(len(stemmedSen)):
          for j in range(len(stemmedSen)):
            if i != j:                         
                sim_mat[i][j] = cosine_similarity(cv_dataframe.loc[i,:].to_numpy().reshape(1,-1),cv_dataframe.loc[j,:].to_numpy().reshape(1,-1))[0][0]
    
    norm = np.sum(sim_mat, axis=1)    
    sim_mat = np.divide(sim_mat, norm, where=norm!=0)
    
    return sim_mat

In [73]:
def TextRankSummarize(text,k=5, alg = "trsim"):
    
    sentences = sent_tokenize(text)
    
    clearedSen = []
    for i in sentences:
        sent= re.sub('[^a-zA-ZÂâğüşöçıİĞÜŞÖÇ]', " ", i)
        sent = sent.lower()
        clearedSen.append(sent)
        
        
    stemmedSen = []

    for i in clearedSen:
        stem = ""
        for j in i.split():
            res = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=str(j)))
            if res.results[0].best.dictionaryItem.lemma.lower() not in stopwords.words('turkish') and res.results[0].best.dictionaryItem.lemma.lower() != 'unk' and len(res.results[0].best.dictionaryItem.lemma.lower()) != 1  :
                stem += res.results[0].best.dictionaryItem.lemma.lower() + " "
            elif res.results[0].best.dictionaryItem.lemma.lower() == 'unk':
                stem += j + " "
        stem.strip()
        stemmedSen.append(stem)
        
    
    if alg == "trsim":
    
        sim_mat = np.zeros([len(stemmedSen), len(stemmedSen)])
        for i in range(len(stemmedSen)):
              for j in range(len(stemmedSen)):
                if i != j:                         
                    sim_mat[i][j] = textRankSimilarities(stemmedSen[i],stemmedSen[j])

                
    elif alg == "cv":
        sim_mat = cvtfidfCosineSim(stemmedSen,alg = "cv")
    elif alg == "tfidf":
        sim_mat = cvtfidfCosineSim(stemmedSen,alg = "tfidf")
    
    #PageRank
    G = nx.from_numpy_array(np.array(sim_mat))       
    scores = nx.pagerank_numpy(G)
    
    

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    my_list=[]
    for i in range(k):
        my_list.append(ranked_sentences[i][1])

    for i in sentences:
        if i in my_list:
            print(i)

In [74]:
txt ="""Bir iklim bilimcinin açıklamasına göre, aşırı küresel sıcaklıklar insan vücudunu termal sınırlara doğru itiyor.

Geçtiğimiz hafta Avrupa genelinde rekor sıcaklıklara şahit olundu. Birkaç ülkede sıcaklık 40 derecenin üstüne çıktı. Öte yandan Güney Asya ve Basra Körfezi gibi yerlerde yaşayan insanlar 54 derecelere varan sıcaklıklarla mücadele ediyor. Loughborough Üniversitesi’nde iklim bilimci olarak çalışan Dr. Tom Matthews “İnsan vücudundaki tüm ısıl verime rağmen Güney Asya ve Basra Körfezi gibi yerler yakın zamanda yaşanmaz hale gelebilir.” dedi.

Hava sıcaklığı 35 dereceyi geçtiğinde vücut, vücut ısısını güvenli seviyede tutmak için terler. Ancak nemin buharlaşabilme yetisini gösteren ıslak sıcaklık 35 dereceyi aştığında bu sistem çalışmaz hale gelir. Dr. Matthews “Islak sıcaklık, termometreden buharlaşan suyun soğuma etkisini de içerir. Bu yüzden doğal olarak hava durumlarında verilen normal (kuru ampul sıcaklığı) sıcaklıktan daha düşük olur. Bu ıslak sıcaklık eşiği aşıldığında, hava o kadar çok su buharıyla dolar ki ter buharlaşmaz hale gelir.” diyor.

Bu da, insan vücudunun kendini birkaç saatten fazla hayatta kalabilecek kadar soğutamayacağı anlamına geliyor. Matthews’a göre 21. yüzyılın sonunda Dünya üzerindeki en yoğun nüfuslu yerlerden bazıları da dahil olmak üzere kimi bölgeler bu eşiği geçecek. İklim değişikliğinin hava sistemlerini derinden etkilemeye başlamasıyla, artan sıcaklıklar yakında dünyanın bazı yerlerini yaşanmaz hale getirebilir. Eğer elektrik sistemi düzgün işletilirse kronik şekilde sıcaktan etkilenen ülkelerde yaşamak mümkün olabilir ancak elektrik kesintisi olması durumunda felaket yaşanır. Bu yüzden Dr. Matthews’a göre konuyla ilgili en sağlıklı hamle, Paris İklim Değişikliği Anlaşması’yla belirlenen sınırları korumak adına seragazı salınımını kesmek."""


In [75]:

TextRankSummarize(txt,5, alg = "trsim" )

Loughborough Üniversitesi’nde iklim bilimci olarak çalışan Dr. Tom Matthews “İnsan vücudundaki tüm ısıl verime rağmen Güney Asya ve Basra Körfezi gibi yerler yakın zamanda yaşanmaz hale gelebilir.” dedi.
Ancak nemin buharlaşabilme yetisini gösteren ıslak sıcaklık 35 dereceyi aştığında bu sistem çalışmaz hale gelir.
Bu ıslak sıcaklık eşiği aşıldığında, hava o kadar çok su buharıyla dolar ki ter buharlaşmaz hale gelir.” diyor.
İklim değişikliğinin hava sistemlerini derinden etkilemeye başlamasıyla, artan sıcaklıklar yakında dünyanın bazı yerlerini yaşanmaz hale getirebilir.
Eğer elektrik sistemi düzgün işletilirse kronik şekilde sıcaktan etkilenen ülkelerde yaşamak mümkün olabilir ancak elektrik kesintisi olması durumunda felaket yaşanır.


In [76]:
TextRankSummarize(txt,5, alg = "cv" )

Loughborough Üniversitesi’nde iklim bilimci olarak çalışan Dr. Tom Matthews “İnsan vücudundaki tüm ısıl verime rağmen Güney Asya ve Basra Körfezi gibi yerler yakın zamanda yaşanmaz hale gelebilir.” dedi.
Hava sıcaklığı 35 dereceyi geçtiğinde vücut, vücut ısısını güvenli seviyede tutmak için terler.
Ancak nemin buharlaşabilme yetisini gösteren ıslak sıcaklık 35 dereceyi aştığında bu sistem çalışmaz hale gelir.
Bu ıslak sıcaklık eşiği aşıldığında, hava o kadar çok su buharıyla dolar ki ter buharlaşmaz hale gelir.” diyor.
İklim değişikliğinin hava sistemlerini derinden etkilemeye başlamasıyla, artan sıcaklıklar yakında dünyanın bazı yerlerini yaşanmaz hale getirebilir.


In [77]:
TextRankSummarize(txt,5, alg = "tfidf" )

Loughborough Üniversitesi’nde iklim bilimci olarak çalışan Dr. Tom Matthews “İnsan vücudundaki tüm ısıl verime rağmen Güney Asya ve Basra Körfezi gibi yerler yakın zamanda yaşanmaz hale gelebilir.” dedi.
Hava sıcaklığı 35 dereceyi geçtiğinde vücut, vücut ısısını güvenli seviyede tutmak için terler.
Ancak nemin buharlaşabilme yetisini gösteren ıslak sıcaklık 35 dereceyi aştığında bu sistem çalışmaz hale gelir.
Bu ıslak sıcaklık eşiği aşıldığında, hava o kadar çok su buharıyla dolar ki ter buharlaşmaz hale gelir.” diyor.
İklim değişikliğinin hava sistemlerini derinden etkilemeye başlamasıyla, artan sıcaklıklar yakında dünyanın bazı yerlerini yaşanmaz hale getirebilir.
